<a id="TableOfContents"></a>
# TABLE OF CONTENTS:
<li><a href='#imports'>Imports</a></li>
<li><a href="#acquire">Acquire</a></li>
<li><a href="#prepare">Prepare</a></li>
<li><a href="#explore">Explore</a></li>
<li><a href="#models">Top 3 Models</a></li>
<li><a href="#conclusion">Conclusion</a></li>

##### Orientation:
- Objective
    - Find drivers for customer churn at Telco
- Method
    - Using telco data from MySQL
    - Clean data for use in jupyter notebook
    - Explore data for any significant features
    - Create model to predict customer churn

<a id='imports'></a>
# Imports:
<li><a href='#TableOfContents'>Table of Contents</a></li>

<a id='acquire'></a>
# Acquire:
<li><a href='#TableOfContents'>Table of Contents</a></li>

### Methodology:
1. Using MySQL
    - Check dataframe and tables
    - Combine all tables with a single query
2. Using Python
    - Create SQL connection URL
    - Copy SQL query
3. Acquire telco data
    - pd.read_sql(query, url)

<a id='prepare'></a>
# Prepare:
<li><a href='#TableOfContents'>Table of Contents</a></li>

<a id='explore'></a>
# Explore:
<li><a href='#TableOfContents'>Table of Contents</a></li>

<a id='models'></a>
# Top 3 Models:
<li><a href='#TableOfContents'>Table of Contents</a></li>

<a id='conclusion'></a>
# Conclusion:
<li><a href='#TableOfContents'>Table of Contents</a></li>

### Recommendations
- Focus advertisement efforts towards families and couples and avoid older population
- Market offers as single services or bundled services with at least 3 additional services
- For customers, establish whether or not they desire additional internet services
    - If yes, push bundles of at least 3 additional services
    - If no, do not allow additional services between 1 and 2
- For paying customers:
    - Do not allow monthly contracts
    - Do not allow electronic check forms of payment
    
### Takeaways
- Target audience:
    - Couples
    - Families/Dependents
- Services Emphasis:
    - Cell Services:
        - Not much difference with/without/multiple
    - Internet Services:
        - Either no internet or at least bundle 3 subs
- Payments:
    - DO NOT ALLOW ELECTRONIC CHECK
- Contracts:
    - DO NOT ALLOW MONTH CONTRACT